In [1]:
import tensorflow as tf
tf.__version__

'2.0.0-beta1'

In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load('imdb_reviews', with_info = True, as_supervised = True)

W0807 23:08:15.966603 140735858033536 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_datasets/core/file_format_adapter.py:209: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /Users/amankedia/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


W0807 23:08:57.903284 140735858033536 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [3]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [4]:
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []

In [5]:
for s, l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

In [6]:
for s, l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

In [7]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [8]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)

padded = pad_sequences(sequences, maxlen = max_length, padding = trunc_type)

In [11]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length, padding = trunc_type)

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [15]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

W0807 23:30:49.450124 140735858033536 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 3s 121us/sample - loss: 0.4686 - accuracy: 0.7590 - val_loss: 0.3287 - val_accuracy: 0.8553
Epoch 2/10
25000/25000 [==============================] - 2s 94us/sample - loss: 0.2081 - accuracy: 0.9202 - val_loss: 0.3487 - val_accuracy: 0.8507
Epoch 3/10
25000/25000 [==============================] - 2s 96us/sample - loss: 0.0757 - accuracy: 0.9815 - val_loss: 0.4241 - val_accuracy: 0.8382
Epoch 4/10
25000/25000 [==============================] - 2s 97us/sample - loss: 0.0234 - accuracy: 0.9970 - val_loss: 0.4864 - val_accuracy: 0.8402
Epoch 5/10
25000/25000 [==============================] - 2s 94us/sample - loss: 0.0089 - accuracy: 0.9989 - val_loss: 0.5382 - val_accuracy: 0.8386
Epoch 6/10
25000/25000 [==============================] - 2s 89us/sample - loss: 0.0040 - accuracy: 0.9995 - val_loss: 0.6019 - val_accuracy: 0.8350
Epoch 7/10
25000/25000 [==============================]

In [16]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [18]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

predictable and but so what i believed them his stress and inability to cope her brave yet slightly misguided attempts to bring them closer together i think the characters acted by anyone else would indeed have been annoying and unbelievable but they weren't it reflects the <OOV> of the situation they're in that he's sitting in class and she walks on campus with the baby i felt angry at her for that i felt angry at him for being such a child and for <OOV> her i felt it all br br in the end i loved it and would recommend it br br watch out for the scene where <OOV> <OOV> runs from the disastrous <OOV> session career performance
b"I saw this film on True Movies (which automatically made me sceptical) but actually - it was good. Why? Not because of the amazing plot twists or breathtaking dialogue (of which there is little) but because actually, despite what people say I thought the film was accurate in it's depiction of teenagers dealing with pregnancy.<br /><br />It's NOT Dawson's Creek,

In [21]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()